In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, dok_matrix
from math import ceil
import sys
from tqdm import trange
from itertools import islice
from sklearn.preprocessing import normalize
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import roc_auc_score

In [2]:
columns = ['user_id', 'item_id', 'rating', 'timestamp']
ratings = pd.read_csv('../ml-100k/u.data', sep='\t', names=columns)
ratings.drop('timestamp', axis=1, inplace=True)

columns = ['item_id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('../ml-100k/u.item', sep='|', names=columns, encoding='latin-1')
movies = movies[['item_id', 'movie title']]

n_users = len(ratings['user_id'].unique())
n_items = len(movies['item_id'].unique())
rating_mat = np.zeros(shape= (n_users, n_items))

In [3]:
ratings.head()

,user_id,item_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [4]:
ratings.shape

(100000, 3)

In [5]:
def create_mat(data, users_col, items_col, ratings_col, threshold=None):
    if threshold is not None:
        data = data[data[ratings_col] >= threshold]
        data[ratings_col] = 1
    data_user_num_items = (data
                           .groupby('user_id')
                           .agg(**{'num_items': ('item_id', 'count')})
                           .reset_index())
    data = data.merge(data_user_num_items, on='user_id', how='inner')
    data = data[data['num_items'] > 1]

    for col in (items_col, users_col, ratings_col):
        data[col] = data[col].astype('category')

    ratings = csr_matrix((data[ratings_col],
                          (data[users_col].cat.codes, data[items_col].cat.codes)))
    ratings.eliminate_zeros()
    return ratings, data

In [6]:
items_col = 'item_id'
users_col = 'user_id'
ratings_col = 'rating'
threshold = 3
X, df = create_mat(ratings, users_col, items_col, ratings_col, threshold)

C:\Users\Yungi\AppData\Local\Temp\ipykernel_120488\4113105480.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[ratings_col] = 1


In [7]:
print(X)

  (0, 0)	1
  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 8)	1
  (0, 9)	1
  (0, 11)	1
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (0, 15)	1
  (0, 16)	1
  (0, 17)	1
  (0, 18)	1
  (0, 19)	1
  (0, 21)	1
  (0, 22)	1
  (0, 23)	1
  (0, 24)	1
  (0, 25)	1
  (0, 27)	1
  (0, 29)	1
  :	:
  (942, 618)	1
  (942, 648)	1
  (942, 665)	1
  (942, 678)	1
  (942, 710)	1
  (942, 714)	1
  (942, 715)	1
  (942, 725)	1
  (942, 732)	1
  (942, 756)	1
  (942, 758)	1
  (942, 785)	1
  (942, 787)	1
  (942, 799)	1
  (942, 807)	1
  (942, 815)	1
  (942, 816)	1
  (942, 830)	1
  (942, 915)	1
  (942, 930)	1
  (942, 1031)	1
  (942, 1061)	1
  (942, 1175)	1
  (942, 1215)	1
  (942, 1308)	1


In [8]:
def create_train_test(ratings, test_size=0.2, seed=1234):
    assert test_size < 1.0 and test_size > 0.0

    train = ratings.copy().todok()
    test = dok_matrix(train.shape)

    rstate = np.random.RandomState(seed)
    for u in range(ratings.shape[0]):
        split_index = ratings[u].indices
        n_splits = ceil(test_size * split_index.shape[0])
        test_index = rstate.choice(split_index, size=n_splits, replace=False)
        test[u, test_index] = ratings[u, test_index]
        train[u, test_index] = 0

    train, test = train.tocsr(), test.tocsr()
    return train, test

In [9]:
X_train, X_test = create_train_test(X, test_size = 0.2, seed=1234)
X_train

<943x1574 sparse matrix of type '<class 'numpy.int64'>'
	with 65641 stored elements in Compressed Sparse Row format>

In [10]:
print(X_train)

  (0, 0)	1
  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 9)	1
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (0, 15)	1
  (0, 16)	1
  (0, 17)	1
  (0, 18)	1
  (0, 19)	1
  (0, 21)	1
  (0, 22)	1
  (0, 23)	1
  (0, 24)	1
  (0, 25)	1
  (0, 27)	1
  (0, 29)	1
  (0, 30)	1
  (0, 31)	1
  (0, 32)	1
  (0, 37)	1
  :	:
  (942, 535)	1
  (942, 540)	1
  (942, 553)	1
  (942, 560)	1
  (942, 562)	1
  (942, 570)	1
  (942, 607)	1
  (942, 618)	1
  (942, 648)	1
  (942, 665)	1
  (942, 678)	1
  (942, 710)	1
  (942, 714)	1
  (942, 715)	1
  (942, 725)	1
  (942, 732)	1
  (942, 756)	1
  (942, 758)	1
  (942, 785)	1
  (942, 787)	1
  (942, 815)	1
  (942, 816)	1
  (942, 830)	1
  (942, 915)	1
  (942, 1175)	1


In [11]:
print(X_test)

  (0, 5)	1.0
  (0, 129)	1.0
  (0, 6)	1.0
  (0, 200)	1.0
  (0, 70)	1.0
  (0, 65)	1.0
  (0, 188)	1.0
  (0, 90)	1.0
  (0, 225)	1.0
  (0, 11)	1.0
  (0, 61)	1.0
  (0, 164)	1.0
  (0, 8)	1.0
  (0, 261)	1.0
  (0, 205)	1.0
  (0, 256)	1.0
  (0, 38)	1.0
  (0, 87)	1.0
  (0, 177)	1.0
  (0, 162)	1.0
  (0, 75)	1.0
  (0, 157)	1.0
  (0, 124)	1.0
  (0, 112)	1.0
  (0, 41)	1.0
  :	:
  (942, 99)	1.0
  (942, 575)	1.0
  (942, 215)	1.0
  (942, 401)	1.0
  (942, 238)	1.0
  (942, 799)	1.0
  (942, 66)	1.0
  (942, 1308)	1.0
  (942, 316)	1.0
  (942, 233)	1.0
  (942, 185)	1.0
  (942, 187)	1.0
  (942, 1215)	1.0
  (942, 807)	1.0
  (942, 1061)	1.0
  (942, 424)	1.0
  (942, 1031)	1.0
  (942, 203)	1.0
  (942, 30)	1.0
  (942, 193)	1.0
  (942, 281)	1.0
  (942, 930)	1.0
  (942, 204)	1.0
  (942, 150)	1.0
  (942, 232)	1.0


In [12]:
class BPR:
    def __init__(self, learning_rate=0.01, n_factors=15, n_iters=10,
                 batch_size = 1000, reg= 0.01, seed=1234, verbose=True):
        self.reg = reg
        self.seed = seed
        self.verbose = verbose
        self.n_iters = n_iters
        self.n_factors = n_factors
        self.batch_size = batch_size
        self.learning_rate = learning_rate

        self._prediction = None

    def fit(self, ratings):
        indptr = ratings.indptr
        indices = ratings.indices
        n_users, n_items = ratings.shape

        batch_size = self.batch_size
        if n_users < batch_size:
            batch_size = n_users
            sys.stderr.write("WARNING: Batch Size가 Users보다 많습니다."
                             'batch_size를 {}로 바꾸세요\n'.format(n_users))

        batch_iters = n_users // batch_size
        rstate = np.random.RandomState(self.seed)
        self.user_factors = rstate.normal(size=(n_users, self.n_factors))
        self.item_factors = rstate.normal(size=(n_items, self.n_factors))

        loop = range(self.n_iters)
        if self.verbose:
            loop = trange(self.n_iters, desc=self.__class__.__name__)

        for _ in loop:
            for _ in range(batch_iters):
                sampled = self._sample(n_users, n_items, indices, indptr)
                sampled_users, sampled_pos_items, sampled_neg_items = sampled
                self._update(sampled_users, sampled_pos_items, sampled_neg_items)
        return self

    def _sample(self, n_users, n_items, indices, indptr):
        sampled_pos_items = np.zeros(self.batch_size, dtype=np.int32)
        sampled_neg_items = np.zeros(self.batch_size, dtype=np.int32)
        sampled_users = np.random.choice(
            n_users, size=self.batch_size, replace=False)

        for idx, user in enumerate(sampled_users):
            pos_items = indices[indptr[user] : indptr[user+1]]
            pos_item = np.random.choice(pos_items)
            neg_item = np.random.choice(n_items)
            while neg_item in pos_items:
                neg_item = np.random.choice(n_items)
            sampled_pos_items[idx] = pos_item
            sampled_neg_items[idx] = neg_item

        return sampled_users, sampled_pos_items, sampled_neg_items

    def _update(self, u, i, j):
        user_u = self.user_factors[u]
        item_i = self.item_factors[i]
        item_j = self.item_factors[j]

        # 원래는 Dot Product를 해서 계산하는게 맞는데, 아다마르(hadamard)곱 연산을 통해서 시간을 줄인다. 그냥 element-wise한 계산을 하자는 것 같음
        r_uij = np.sum(user_u * (item_i - item_j), axis = 1)
        sigmoid = np.exp(-r_uij) / (1.0 + np.exp(-r_uij))

        sigmoid_tiled = np.tile(sigmoid, (self.n_factors, 1)).T


        grad_u = sigmoid_tiled * (item_j - item_i) + self.reg * user_u
        grad_i = sigmoid_tiled * -user_u + self.reg * item_i
        grad_j = sigmoid_tiled * user_u + self.reg * item_j
        self.user_factors[u] -= self.learning_rate * grad_u
        self.item_factors[i] -= self.learning_rate * grad_i
        self.item_factors[j] -= self.learning_rate * grad_j
        return self

    def predict(self):
        if self._prediction is None:
            self._prediction = self.user_factors.dot(self.item_factors.T)

        return self._prediction

    def _predict_user(self, user):
        user_pred = self.user_factors[user].dot(self.item_factors.T)
        return user_pred

    def recommend(self, ratings, N=5):
        n_users = ratings.shape[0]
        recommendation = np.zeros((n_users, N), dtype=np.uint32)
        for user in range(n_users):
            top_n = self._recommend_user(ratings, user, N)
            recommendation[user] = top_n

        return recommendation

    def _recommend_user(self, ratings, user, N):
        scores = self._predict_user(user)

        liked = set(ratings[user].indices)
        count = N + len(liked)
        if count < scores.shape[0]:
            ids = np.argpartition(scores, -count)[-count:]
            best_ids = np.argsort(scores[ids])[::-1]
            best = ids[best_ids]
        else:
            best = np.argsort(scores)[::-1]

        top_n = list(islice((rec for rec in best if rec not in liked), N))
        return top_n

    def get_similar_items(self, N=5, item_ids = None):
        normed_factors = normalize(self.item_factors)
        knn = NearestNeighbors(n_neighbors = N + 1, metric = 'euclidean')
        knn.fit(normed_factors)

        # returns a distance, index tuple,
        # we don't actually need the distance
        if item_ids is not None:
            normed_factors = normed_factors[item_ids]

        _, items = knn.kneighbors(normed_factors)
        similar_items = items[:, 1:].astype(np.uint32)
        return similar_items

In [13]:
bpr_params= {
    'reg': 0.01,
    "learning_rate": 0.1,
    'n_iters': 160,
    'n_factors': 15,
    'batch_size': 100
}
bpr = BPR(**bpr_params)
bpr.fit(X_train)

BPR: 100%|██████████| 160/160 [00:05<00:00, 28.46it/s]


In [14]:
def auc_score(model, ratings):
    auc = 0.0
    n_users, n_items = ratings.shape
    for user, row in enumerate(ratings):
        y_pred = model._predict_user(user)
        y_true = np.zeros(n_items)
        y_true[row.indices] = 1
        auc += roc_auc_score(y_true, y_pred)

    auc /= n_users
    return auc

In [15]:
print(auc_score(bpr, X_train))
print(auc_score(bpr, X_test))

0.8978349211141912
0.8296478910149392


In [16]:
bpr.get_similar_items(N=5)

array([[ 110,   24,   49,  180,  403],
       [ 130,  687,  425, 1384,  135],
       [1348,  278,  895,   27, 1007],
       ...,
       [1494,  661,  951,  955,  447],
       [ 366, 1115, 1302,  351,  863],
       [ 809,  891, 1084,  750,  923]], dtype=uint32)

In [17]:
bpr.recommend(X_train, N=10)

array([[492, 193, 508, ..., 182, 685, 425],
       [  8, 741, 806, ..., 733, 267,  99],
       [683, 301, 320, ..., 675, 313, 356],
       ...,
       [219, 469,  23, ...,  13, 117, 868],
       [172, 763, 162, ..., 285,   0, 167],
       [167, 184, 728, ..., 237,   6, 264]], dtype=uint32)